In [ ]:
import pandas as pd
# df = pd.read_csv("universities_osm.csv")
# df = df.drop('geometry', axis=1)
# df.sample(10)

# df.to_csv("final.csv")


# phar = pd.read_csv("pharmacy_osm.csv")
# phar = phar.drop("geometry", axis=1)
# phar.head()

# phar.to_csv("final_pharmacy.csv")



rest = pd.read_csv("restaurants_osm.csv")
rest = rest.drop("geometry", axis = 1)
rest.head()
rest.to_csv("final_restaurant.csv")


In [ ]:
import pandas as pd
import folium
import geopandas as gpd
import osmnx as ox
import numpy as np
from shapely.geometry import box

# -------------------------------
# 1. Load Islamabad boundary and create grid
# -------------------------------
place_name = "Islamabad, Pakistan"
boundary = ox.geocode_to_gdf(place_name)

minx, miny, maxx, maxy = boundary.total_bounds
step = 0.009

cells = [box(x, y, x + step, y + step)
         for x in np.arange(minx, maxx, step)
         for y in np.arange(miny, maxy, step)]

grid = gpd.GeoDataFrame(geometry=cells, crs="EPSG:4326")
clipped_grid = gpd.overlay(grid, boundary, how="intersection")
clipped_grid["grid_id"] = range(len(clipped_grid))

# -------------------------------
# 2. Load university data from CSV
# -------------------------------
df_uni = pd.read_csv("final.csv")
df_phar = pd.read_csv("final_pharmacy.csv")
df_rest = pd.read_csv("final_restaurant.csv")
# -------------------------------
# 3. Initialize folium map
# -------------------------------
center = boundary.geometry.centroid.iloc[0]
m = folium.Map(location=[center.y, center.x], zoom_start=12, tiles="OpenStreetMap")

# -------------------------------
# 4. Add grid cells
# -------------------------------
for _, row in clipped_grid.iterrows():
    folium.GeoJson(
        row["geometry"],
        style_function=lambda x: {
            'fillColor': 'none',
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0
        },
        tooltip=f"Grid ID: {row['grid_id']}"
    ).add_to(m)

# -------------------------------
# 5. Add university markers with flag icon
# -------------------------------
for _, row in df_uni.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=row['name'],
        tooltip=row['name'],
        icon=folium.Icon(icon='flag', color='blue')
    ).add_to(m)


# FOR THE PHARMACY;

for _, row in df_phar.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=row['name'],
        tooltip=row['name'],
        icon=folium.Icon(icon='flag', color='red')
    ).add_to(m)

# FOR THE RESTAURANTS

for _, row in df_rest.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=row['name'],
        tooltip=row['name'],
        icon=folium.Icon(icon='flag', color='green')
    ).add_to(m)


import os

if os.path.exists("C:/Users/tehma/OneDrive/Desktop/Shop Prediction Model/SHOP_PROJECT/All_HTML_Mappings/islamabad_grid_with_universities.html"):
    # File exists, proceed with deletion
    pass
else:
    print("Nothing")


m.save("C:/Users/tehma/OneDrive/Desktop/Shop Prediction Model/SHOP_PROJECT/All_HTML_Mappings/islamabad_grid_with_universities.html")



In [ ]:
m

In [ ]:
import pandas as pd
import googlemaps
import time

# Load your pharmacy dataset
df = pd.read_csv("final_pharmacy.csv")

API_KEY = os.getenv("key")
gmaps = googlemaps.Client(key=API_KEY)

# Prepare columns
df["rating"] = None
df["user_ratings_total"] = None

# Search for each pharmacy using name and location
for idx, row in df.iterrows():
    name = row["name"]
    lat = row["lat"]
    lon = row["lon"]
    
    try:
        # Search with name and location bias
        result = gmaps.places(query=name, location=(lat, lon), radius=100)
        if result["results"]:
            top = result["results"][0]
            df.at[idx, "rating"] = top.get("rating")
            df.at[idx, "user_ratings_total"] = top.get("user_ratings_total")
        
        time.sleep(1)  # Be polite to the API
    except Exception as e:
        print(f"Error at index {idx}: {e}")
        continue

# Save the enriched dataset
df.to_csv("most_final_pharmacies.csv", index=False)
print("✅ Done! Saved to 'pharmacies_with_ratings.csv'")


import pandas as pd

new_df = pd.read_csv("most_final_pharmacies.csv")

new_df.head()

In [ ]:
new_df.describe()

In [ ]:
new_df.sample(10)